In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates

# validation modules
import experiment
import metrics

# requires cleanair
sys.path.append("../containers")
from cleanair.models import ModelData

In [2]:
# read experiment from file
name = 'basic'
cluster = 'laptop'
exp = experiment.experiment_from_dir(name, cluster)

In [3]:
# load the data_model objects
secret_fp = "../terraform/.secrets/db_secrets.json"
model_data_list = []
for data_config in exp.data_config:
    model_data = ModelData(config=data_config, secretfile=secret_fp)
    model_data_list.append(model_data)

In [4]:
# get a list of model data objects with updated prediction columns
updated_preds = []


for index, row in exp.experiment_df.iterrows():
    row = exp.experiment_df.loc[index]
    pid = row['param_id']
    model = row['model_name']
    did = row['data_id']
#     y_test = np.load(exp.data_config[did]['y_test_fp'])
    Y_pred = np.load(row['y_pred_fp'])
    
    # look at shapes
    data_config = exp.data_config[did]
    x_test = np.load(data_config['x_test_fp'])
    y_test = np.load(data_config['y_test_fp'])
    print("Xs :", x_test.shape)
    print("Ys :", y_test.shape)
    
    # update model_data
    normalised_df = model_data_list[did].normalised_pred_data_df.copy()
    predict_data_dict = model_data_list[did].get_pred_data_arrays(return_y=True).copy()
    
    # Create new dataframe with predictions
    predict_df = pd.DataFrame(index=predict_data_dict['index'])
    print("Pred index shape:", predict_df.shape)
    print("Y pred shape:", Y_pred[:, 0].shape)
    predict_df['predict_mean'] = Y_pred[:, 0]
    predict_df['predict_var'] = Y_pred[:, 1]
    predict_df['fit_start_time'] = exp.data_config[did]['pred_start_date']
    predict_df['tag'] = 0
    
    normalised_pred_data_df = pd.concat([model_data_list[did].normalised_pred_data_df, predict_df], axis=1, ignore_index=False)
    print(normalised_pred_data_df.sample(3))
    updated_preds.append(normalised_pred_data_df)
    

Xs : (2918, 143)
Ys : (2918, 1)
Pred index shape: (2918, 0)
Y pred shape: (2918,)
                                  point_id measurement_start_utc source  \
4150  af257e35-888d-401e-b67c-d8bf0675f6d7   2019-11-03 22:00:00   laqn   
1549  6609f5b7-7dd6-44f1-bb7b-ff4182e9a18c   2019-11-03 13:00:00   laqn   
1651  edced3c0-5b17-46ed-a877-d59e30d63ed0   2019-11-03 19:00:00   laqn   

                                             location_x       lon        lat  \
4150  0101000020e61000004ccff26f7b4cd0bf96c78f3f7ec0... -0.254668  51.503853   
1549  0101000020e610000021dc0c0dadfdb9bf811ee4b91fbf... -0.101527  51.493156   
1651  0101000020e6100000c51b9947fe60cebf117349d576bd... -0.237335  51.480189   

      value_1000_avg_min_width  value_1000_avg_ratio_avg  \
4150                 11.512792                  0.560596   
1549                 11.207888                  0.794057   
1651                 14.413919                  0.509587   

      value_1000_building_height  value_1000_flat  ... 

[3 rows x 296 columns]


In [5]:
i = 0
preds = updated_preds[0]
preds['measurement_start_utc'] = pd.to_datetime(preds['measurement_start_utc'])
gb = preds.groupby('point_id')
print(preds.columns)

Index(['point_id', 'measurement_start_utc', 'source', 'location_x', 'lon',
       'lat', 'value_1000_avg_min_width', 'value_1000_avg_ratio_avg',
       'value_1000_building_height', 'value_1000_flat',
       ...
       'value_10_max_flow_count_norm', 'value_10_max_occupancy_count_norm',
       'value_10_max_occupancy_percentage_norm',
       'value_10_total_flow_count_norm', 'value_10_total_occupancy_count_norm',
       'value_10_total_occupancy_percentage_norm', 'predict_mean',
       'predict_var', 'fit_start_time', 'tag'],
      dtype='object', length=296)


In [6]:
def plot_pretty_timeseries(time, series, ax, sigmas=None):
    ax.plot_date(time, series, ',', linestyle='-')
    ax.xaxis.grid(True, which="major")
    ax.xaxis.set_major_locator(dates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(dates.DateFormatter('%d\n%m'))
    
    if not sigmas is None:
        factor = 2
        y_min = series - factor*np.sqrt(sigmas)
        y_max = series + factor*np.sqrt(sigmas)
        ax.fill_between(time, y_min, y_max, color='orange', alpha=0.5)
        
        
    return ax

In [7]:
n = 2 # number of rows
c = 2 # number of columns
fig, axs = plt.subplots(n, c, sharex=True, sharey=True)
i = 0
j = 0
for point, row in gb:
    plot_pretty_timeseries(row['measurement_start_utc'], row['predict_mean'], axs[i, j], sigmas=row['predict_var'])
    plot_pretty_timeseries(row['measurement_start_utc'], row['NO2'], axs[i, j])
    i += j % c
    j += 1
    j = j % c
    if i == n:
        break

/Users/pohara/Documents/repos/clean-air-infrastructure/python-virtual-environments/env/lib/python3.6/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning:

Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()



RuntimeError: Locator attempting to generate 7972 ticks from 729758.0 to 737729.0: exceeds Locator.MAXTICKS